In [1]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [2]:
!ls

drive  sample_data


In [ ]:
# Situacao 1: Ticker esta presente na lista de validos, mas nao tem .csv presente na pasta de dados

In [3]:
import pandas as pd
import datetime
import os.path
import pathlib

import re # built-in method for splitting strings
import random

In [4]:
!pip install findatapy

     |████████████████████████████████| 1.7MB 2.7MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 81kB 7.6MB/s 
     |████████████████████████████████| 81kB 7.0MB/s 
     |████████████████████████████████| 491kB 16.6MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 5.5MB 21.8MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 71kB 8.8MB/s 
     |████████████████████████████████| 204kB 47.5MB/s 
     |████████████████████████████████| 1.3MB 42.2MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 
     |████████████████████████████████| 2.6MB 45.6MB/s 
     |████████████████████████████████| 245kB 44.3MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 296kB 43.5MB/s 
     |████████████████████████████████| 143kB 38.3MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 
     |█████

In [5]:
from findatapy.market import Market, MarketDataRequest, MarketDataGenerator


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
#foo = pd.read_csv("/content/drive/My Drive/Finance/data/aapl.csv")
PATH = "/content/drive/My Drive/Finance/data"
YAHOO_VALID_STOCKS_FILE = "/content/drive/My Drive/Finance/data/stocks"

MONTH_CONVERTER = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 
                   5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 
                   9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}


# Functions and Classes

In [66]:
def saveStocksToCSV(df, name, exchange=None):
    if exchange == None:
        filepath = "{}/{}.csv".format(PATH, name)
        output_dir = Path(PATH)
        output_dir = "{}".format(PATH)
    else:
        filepath = "{}/{}/{}.csv".format(PATH, exchange, name)
        output_dir = Path("{}/{}".format(PATH, exchange))
    try:
        output_dir.mkdir(parents=True, exist_ok=True)
        df.to_csv(filepath)
        print("Saved {}.csv successfully".format(name))
    except Exception as e:
        print("Error '{}' occured while saving '{}'".format(e, filepath))


def updateStockData(vendor_ticker):
    # Get date of the most recent data entry from csv file
    df = pd.read_csv(PATH + "/{}.csv".format(vendor_ticker), index_col=0)
    print("Data mais recente: {}".format(df.index[-1]))
    last_date = df.index[-1]
    split_date = re.split('-', last_date)
    year, month, day = int(split_date[0]), int(split_date[1]), int(split_date[2])
    start_date = "{} {} {}".format(MONTH_CONVERTER[month], day, year)

    # Fetch data
    md_request = MarketDataRequest(
        start_date=start_date,
        data_source='yahoo', 
        tickers=vendor_ticker,
        fields=['open', 'high', 'low', 'close', 'adjclose', 'volume'],
        vendor_tickers=vendor_ticker,
        vendor_fields=['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'])

    new_df = market.fetch_market(md_request)

    # "Append" the new data in the csv file
    df = pd.concat([df, new_df], axis=0)
    df.drop_duplicates(inplace=True)
    return df


def getStocksHistoricalData(vendor_ticker, exchange=None):
    # Fetch data
    md_request = MarketDataRequest(
        start_date = 'Jan 1 1950',
        data_source='yahoo', 
        tickers=vendor_ticker,
        fields=['open', 'high', 'low', 'close', 'adjclose', 'volume'],
        vendor_tickers=vendor_ticker,
        vendor_fields=['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'])

    df = market.fetch_market(md_request)
    return df


def saveStocksHistoricalData(vendor_ticker, exchange=None):
   # Fetch data
    md_request = MarketDataRequest(
        start_date = 'Jan 1 1950',
        data_source='yahoo', 
        tickers=vendor_ticker,
        fields=['open', 'high', 'low', 'close', 'adjclose', 'volume'],
        vendor_tickers=vendor_ticker,
        vendor_fields=['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'])

    df = market.fetch_market(md_request)
    saveStocksToCSV(df=df, name=vendor_ticker, exchange=exchange)


def addToYahooTickerList(ticker, valid, exchange=None):
    stock_list = "yahoo_valid_stocks.csv" if valid else "yahoo_invalid_stocks.csv"
    aux = "valid" if valid else "invalid"
    f_path = "{}/{}".format(YAHOO_VALID_STOCKS_FILE, stock_list)
    
    if os.path.isfile(f_path):
        df = pd.read_csv(f_path, index_col=0)
        if ticker not in df.index.values:
            new_entry = pd.DataFrame([[ticker, exchange]], columns=['ticker', 'exchange']).set_index('ticker')
            df = df.append(new_entry)
            df.to_csv(f_path, mode='w', index=True)
            print("Ticker {} 'added' to {} tickers. (not verified)".format(ticker, aux))
        else:
            print("Ticker {} is already present in {} tickers.".format(ticker, aux))
    else:
        try:
            pd.DataFrame(columns=['ticker', 'exchange']).set_index('ticker').to_csv(f_path, index=True)
            print ("File not exist. New file created.")
            addToYahooTickerList(ticker, valid, exchange)
        except:
            print ("File not exist. Could not create new file.")


In [44]:
def removeFromYahooTickerList(ticker, valid, exchange=None):
    stock_list = "yahoo_valid_stocks.csv" if valid else "yahoo_invalid_stocks.csv"
    aux = "valid" if valid else "invalid"
    f_path = "{}/{}".format(YAHOO_VALID_STOCKS_FILE, stock_list)
    
    if os.path.isfile(f_path):
        df = pd.read_csv(f_path, index_col=0)
        if ticker in df.index.values:
            #new_entry = pd.DataFrame([[ticker, None]], columns=['ticker', 'exchange']).set_index('ticker')
            #df = df.append(new_entry)
            df.drop(index=ticker, inplace=True)
            df.to_csv(f_path, mode='w', index=True)
            print("Ticker {} 'removed' to {} tickers. (not verified)".format(ticker, aux))
        else:
            print("Ticker {} is not present in {} tickers.".format(ticker, aux))

    else:
        try:
            pd.DataFrame(columns=['ticker', 'exchange']).set_index('ticker').to_csv(f_path, index=True)
            print ("File not exist. New file created.")
            addToYahooTickerList(ticker, valid, exchange)
        except:
            print ("File not exist. Could not create new file.")

      
def verifyYahooTickerValidity(tickerList, exchange=None):
  for vendor_ticker in tickerList:
      df = getStocksHistoricalData(vendor_ticker, exchange)
      if df.empty:
          addToYahooTickerList(ticker=vendor_ticker, valid=False, exchange=exchange)
      else:
          addToYahooTickerList(ticker=vendor_ticker, valid=True, exchange=exchange)
          saveStocksToCSV(df=df, name=vendor_ticker, exchange=exchange)

In [78]:
update

In [79]:
#verifyYahooTickerValidity(vendor_ticker_list, 'ISE')

# Fake "main"

In [10]:

market = Market(market_data_generator=MarketDataGenerator())

#stockDict = [{'ticker': 'Apple', 'vendor_ticker': 'aapl'}]
#ticker = 'Lojas Americanas S.A.' #'Apple'
vendor_ticker = 'aapl'
#ticker = br_stuff['Name'].values.tolist()[:100] #'Apple'
#vendor_ticker = br_stuff['Ticker'].values.tolist()[:100] #'aapl'

md_request = MarketDataRequest(
    start_date='Jun 1 2020 01:33',
    finish_date = 'Nov 2 2020',
    data_source='yahoo',
    #category = "fundamental",
    tickers=vendor_ticker,
    fields=['open', 'high', 'low', 'close', 'adjclose', 'volume'],
    vendor_tickers=vendor_ticker,
    vendor_fields=['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'])

df = market.fetch_market(md_request)
#df = pd.read_csv(PATH + "/{}.csv".format(vendor_ticker), index_col=0)
#saveStocksHistoricalData(vendor_ticker, exchange="SAO")

#df_updated = updateStockData(vendor_ticker)
#saveStocksToCSV(df_updated, vendor_ticker)

2020-11-11 17:58:41,340 - findatapy.market.datavendorweb - INFO - Request Yahoo data
2020-11-11 17:58:41,585 - numexpr.utils - INFO - NumExpr defaulting to 2 threads.
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
2020-11-11 17:58:41,776 - findatapy.market.datavendorweb - INFO - Completed request from Yahoo.
2020-11-11 17:58:41,811 - findatapy.market.ioengine - WARNING - Couldn't push MarketDataRequest_466__abstract_curve_key-None__category-None__category_key-backtest_default-cat_yahoo_daily_NYC__cut-NYC__data_source-yahoo__environment-backtest__expiry_date-NaT__fields-open_high_low_close_adjclose_volume__finish_date-2020-11-02 00:00:00__freq-daily__freq_mult-1__gran_freq-None__push_to_cache-True__resample-None__resample_how-last__start_date-2020-06-01 01:33:00__tickers-aapl__trade_side-trade__vendor_fields-Open_High_Low_Close_Adj Close_Volume__vendor_tickers-aapl to Redis: Error 111 connecting t

# Experiment 1

In [ ]:
#datetime.datetime.now().year

2020

In [ ]:
 random.randint(50, 150)

52

In [14]:
stuff_path = "/content/drive/My Drive/Finance/Yahoo Ticker Symbols - September 2017.xlsx"
stuff = pd.read_excel(stuff_path, sheet_name="Stock", header=3, usecols="A:E")
stuff

,Ticker,Name,Exchange,Category Name,Country
0,OEDV,"Osage Exploration and Development, Inc.",PNK,NaN,USA
1,AAPL,Apple Inc.,NMS,Electronic Equipment,USA
2,BAC,Bank of America Corporation,NYQ,Money Center Banks,USA
3,AMZN,"Amazon.com, Inc.",NMS,Catalog & Mail Order Houses,USA
4,T,AT&T Inc.,NYQ,Telecom Services - Domestic,USA
...,...,...,...,...,...
106323,G5N.SG,"Groupon Inc. Reg. Shares DL-,00",STU,NaN,Germany
106324,N1H.AX,N1 Holdings Limited,ASX,Mortgage Investment,Australia
106325,1K2.BE,"K2M GROUP HLDGS DL-,001",BER,NaN,Germany
106326,2RM.BE,"RE/MAX HLDGS A DL -,0001",BER,NaN,Germany


In [57]:
#stuff['Exchange'].value_counts().tail(10)
stuff[stuff['Exchange'] == 'ISE']

,Ticker,Name,Exchange,Category Name,Country
1581,AIB.IR,AIB.IR,ISE,NaN,Ireland
6056,DQ5.IR,CPL Resources plc,ISE,NaN,Ireland
13000,JEV.IR,JEV.IR,ISE,NaN,Ireland
13288,IRES.IR,Irish Residential Properties REIT plc,ISE,NaN,Ireland
13557,MSTY.IR,Mainstay Medical International plc,ISE,NaN,Ireland
14015,SK3.IR,Smurfit Kappa Group plc,ISE,NaN,Ireland
14412,KRZ.IR,Kerry Group plc,ISE,NaN,Ireland
17963,FAC.IR,Falcon Oil & Gas Ltd.,ISE,NaN,Ireland
18065,BIR.IR,BANK OF IRELAND ORD STK EUR0.05,ISE,NaN,Ireland
18527,ORQ.IR,Ormonde Mining plc,ISE,NaN,Ireland


In [77]:
br_stuff = stuff[stuff['Exchange'] == 'ISE'] #SAO
ticker = br_stuff['Name'].values.tolist() #'Apple'
vendor_ticker_list = br_stuff['Ticker'].values.tolist()[:100] #'aapl'
vendor_ticker_list

['AIB.IR',
 'DQ5.IR',
 'JEV.IR',
 'IRES.IR',
 'MSTY.IR',
 'SK3.IR',
 'KRZ.IR',
 'FAC.IR',
 'BIR.IR',
 'ORQ.IR',
 'T7O.IR',
 'MLC.IR',
 'APGN.IR',
 'HSW.IR',
 'YZA.IR',
 'AIB1.IR',
 'RY4C.IR',
 'ZAZ.IR',
 'FKV1.IR',
 'DOY.IR',
 'DOP.IR',
 'PPB.IR',
 'DD7E.IR',
 'DD7D.IR',
 'GN1.IR',
 'CRG.IR',
 'CRO.IR',
 'DD8B.IR',
 'DD8A.IR',
 'DLE.IR',
 'DHG.IR',
 'WLG.IR',
 'KJY.IR',
 'DQ7.IR',
 'EG7.IR',
 'EG5A.IR',
 'FQ3.IR',
 'GAME.IR',
 'GCC.IR',
 'HBRN.IR',
 'GL9.IR',
 'OVXA.IR',
 'GYQ.IR',
 'P8ET.IR',
 'IPDC.IR',
 '3MP.IR',
 'KRX.IR',
 '8GW.IR',
 'OIZ.IR',
 'PZQA.IR',
 'IL0A.IR',
 'IJG.IR',
 'IR5B.IR',
 'MIO.IR',
 'TQW.IR']

In [12]:
df

,aapl.open,aapl.high,aapl.low,aapl.close,aapl.adjclose,aapl.volume
Date,,,,,,
2020-06-02,80.187500,80.860001,79.732498,80.834999,80.550545,87642800.0
2020-06-03,81.165001,81.550003,80.574997,81.279999,80.993980,104491200.0
2020-06-04,81.097504,81.404999,80.195000,80.580002,80.296448,87560400.0
2020-06-05,80.837502,82.937500,80.807503,82.875000,82.583374,137250400.0
2020-06-08,82.562500,83.400002,81.830002,83.364998,83.071640,95654400.0
...,...,...,...,...,...,...
2020-10-26,114.010002,116.550003,112.879997,115.050003,114.851852,111850704.0
2020-10-27,115.489998,117.279999,114.540001,116.599998,116.399178,92276800.0
2020-10-28,115.050003,115.430000,111.099998,111.199997,111.008476,143937792.0


In [49]:
#vendor_ticker = br_stuff['Ticker'].values.tolist()[:100] #'aapl'
#vendor_ticker
file_foo = "{}/{}.csv".format(PATH, "EG5A.IR")
file_bar = "/content/drive/My Drive/Finance/data/TAL/PKG1T.TL.csv"
df.to_csv(file_bar)

FileNotFoundError: ignored

In [54]:
from pathlib import Path

output_file = 'my_file.csv'
output_dir = Path('long_path/to/my_dir')

#output_dir.mkdir(parents=True, exist_ok=True)

#df.to_csv(output_dir / output_file)  # can join path elements with / operator

print("{}/{}".format(output_dir, output_file))


long_path/to/my_dir/my_file.csv


In [76]:
#df = getStocksHistoricalData('TQW.IR', 'ISE')
#saveStocksToCSV(df, 'TQW.IR', 'ISE')
#saveStocksToCSV(df, 'bar.br', 'foo')

